In [1]:
!ls model/

glove.pkl     glove.pt      gloveMixed.pt mixed.pkl


In [2]:
import torch
import glove
import run
import pickle
from tools import Dictionary

In [3]:
EMBEDDING_SIZE = 128
CONTEXT_SIZE = 3
NUM_EPOCH = 100
BATHC_SIZE = 512
PATH = "model/gloveMixed.pt"

In [4]:
with open("data/f.pkl", mode='rb') as fp:
    doc = pickle.load(fp)

In [5]:
dictionary = Dictionary()
dictionary.update(doc)
# doc.vocab_size
VS = dictionary.vocab_size

In [6]:
len(list(dictionary.word2idx.keys()))

21281

In [8]:
model = glove.GloVeMixedCurvature(EMBEDDING_SIZE, CONTEXT_SIZE, VS)
model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))
model

C euc 0.0, hyp -0.5, sph 0.5


GloVeMixedCurvature(
  (euc): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (hyp): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (sph): ManifoldEmbedding(
    (_focal_embeddings): Embedding(21281, 128)
    (_context_embeddings): Embedding(21281, 128)
    (_focal_biases): Embedding(21281, 1)
    (_context_biases): Embedding(21281, 1)
    (manifold): Stereographic manifold
  )
  (_focal_embeddings): Embedding(21281, 128)
  (_context_embeddings): Embedding(21281, 128)
  (_focal_biases): Embedding(21281, 1)
  (_context_biases): Embedding(21281, 1)
)

In [10]:
ws = torch.nn.functional.softmax(model.w)

curvatures = {
    'euc': float(model.euc.manifold.k.data.detach().numpy()),
    'hyp': float(model.hyp.manifold.k.data.detach().numpy()),
    'sph': float(model.sph.manifold.k.data.detach().numpy()),
} 

weights = {
    'euc': ws.detach().numpy()[0],
    'hyp': ws.detach().numpy()[1],
    'sph': ws.detach().numpy()[2],
} 

print(curvatures)
print(weights)

{'euc': 0.058570027351379395, 'hyp': 0.5209340453147888, 'sph': -0.28635114431381226}
{'euc': 6.9790296e-10, 'hyp': 5.0195154e-10, 'sph': 1.0}


/var/folders/69/w6ds_szn4333rz0srzmb5drh0000gn/T/ipykernel_89426/2681497438.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  ws = torch.nn.functional.softmax(model.w)


In [12]:
emb = [
    model.euc._focal_embeddings.weight + model.euc._context_embeddings.weight,
    model.hyp._focal_embeddings.weight + model.hyp._context_embeddings.weight,
    model.sph._focal_embeddings.weight + model.sph._context_embeddings.weight,
]
emb = [e.detach().numpy() for e in emb]

In [13]:
emb[0].shape

(21281, 128)

In [14]:
mixedGlove = {}
for w in dictionary.word2idx:
    mixedGlove[w] = {
        "euc":emb[0][dictionary.word2idx[w]],
        "hyp":emb[1][dictionary.word2idx[w]],
        "sph":emb[2][dictionary.word2idx[w]],
    }

In [15]:
embedding = {
    "curvatures": curvatures,
    "weights": weights,
    "vecs": mixedGlove,
}

In [16]:
# embedding

In [17]:
with open("model/mixed.pkl", "wb") as handle:
    pickle.dump(embedding, handle)

# Analogies

In [ ]:
def find_analogy_mix(word_a, word_b, word_c, embeddings_list, curvatures, weights):
    word_a = word_a.lower()
    word_b = word_b.lower()
    word_c = word_c.lower()
    
    final = {}
    words = embeddings_list[0].keys()
    word_dist = []
    parallen_transes = []
    for i in range(len(curvatures)):

        e_a, e_b, e_c = embeddings_list[i][word_a], embeddings_list[i][word_b], embeddings_list[i][word_c]

        parallen_trans = mobmath.mobius_add(e_c, mobmath.gyration(e_c, -e_a, mobmath.mobius_add(-e_a, e_b, k=curvatures[i]), k=curvatures[i]), k=curvatures[i])
        parallen_transes.append(parallen_trans)

    for w in words:
        
        if w in [word_a, word_b, word_c]:
                continue

        dist = 0.0
    
        for i, embeddings in enumerate(embeddings_list):
            e_a, e_b, e_c = embeddings[word_a], embeddings[word_b], embeddings[word_c]

            c = curvatures[i]
            wt = weights[i]
            parallen_trans = parallen_transes[i]

            # cosine_sim = find_distance(embeddings[w], mobmath.mobius_add(e_c, )   ()find_distance(e_b, e_a), find_distance(, e_c))
            # parallen_trans = self.moebius_add_mat(pos_emb[1], self.gyr_mat(pos_emb[1], -neg_emb, self.moebius_add_mat(-neg_emb, pos_emb[0])))

            distance = find_distance(embeddings[w], parallen_trans, c)
            dist += wt * distance
            # if distance > max_cosine_sim:
                # max_cosine_sim = distance
                # best_word = w
            
        word_dist.append((dist, w))

    word_dist = sorted(word_dist)
    final = word_dist[:10]
    return final
